# Herramientas generales para la generación del corpus.

**Investigadores**: <br>
  Dr. Ramón Zatarain Cabada<br>
  Dra. María Lucía Barrón Estrada<br>
  M.C. Víctor Manuel Bátiz Beltrán

## Importación de Bibliotecas

Los siguientes comandos instalarán los paquetes de Python necesarios para grabar fragmentos de audio y utilizar los modelos Whisper para la transcripción de voz a texto.

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

También ocuparemos lo siguiente para poder grabar audio desde este cuaderno y poder procesar los archivos resultantes.

In [ ]:
!apt install ffmpeg
!apt-get install libportaudio2

In [ ]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Realizamos una grabación de prueba

Activaremos algunos widgets que nos proporciona Colab para poder realizar la grabación.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Pulsa el botón circular y empieza a hablar. Puede que no lo parezca, pero el widget estará capturando sonido. Pulsa de nuevo el botón del círculo cuando hayas terminado. El widget empezará inmediatamente a reproducir lo que ha capturado.

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

El formato de audio capturado anteriormente no es legible por PyTorch. En este paso, convertimos nuestra grabación a un formato que PyTorch pueda entender. 

In [ ]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav my_recording.wav -y -hide_banner -loglevel panic

## Opciones de selección

Whisper es capaz de realizar transcripciones para muchos idiomas (aunque funciona mejor para algunos idiomas y peor para otros). 

Whisper también es capaz de detectar el idioma de entrada. Sin embargo, para estar seguros, podemos decirle explícitamente a Whisper qué idioma debe esperar. 

In [ ]:
language_options = whisper.tokenizer.TO_LANGUAGE_CODE 
language_list = list(language_options.keys())

In [ ]:
lang_dropdown = widgets.Dropdown(options=language_list, value='spanish')
output = widgets.Output()
display(lang_dropdown)

Whisper también es capaz de realizar varias tareas, como la transcripción sólo en inglés, la traducción de cualquier idioma al inglés y la transcripción de idiomas distintos del inglés. 

A continuación puede seleccionar «transcripción» (que producirá texto en el mismo idioma que el de entrada) o «traducción» (que transcribirá de no inglés a inglés). 

![Capacidades de Whisper](https://cdn.openai.com/whisper/draft-20220920a/asr-training-data-desktop.svg)

Imagen tomada de [Introducción a Whisper](https://openai.com/blog/whisper/) by OpenAI

In [ ]:
task_dropdown = widgets.Dropdown(options=['transcribe', 'translate'], value='transcribe')
output = widgets.Output()
display(task_dropdown)